In [137]:
!pip install requests --upgrade --quiet

In [1]:
import requests

In [2]:
import pandas as pd

In [3]:
import os

In [43]:
import sqlite3

In [21]:
os.makedirs('data', exist_ok=True)

In [4]:
response = requests.get('https://github.com/topics')

In [5]:
response.status_code

200

In [6]:
page_contents = response.text

In [7]:
!pip install beautifulsoup4 --upgrade --quiet

In [8]:
from bs4 import BeautifulSoup

In [9]:
doc = BeautifulSoup(page_contents, 'html.parser')

In [10]:
titles_tags = doc.find_all('p', {'class' : "f3 lh-condensed mb-0 mt-1 Link--primary"})

In [11]:
titles = []
for i in titles_tags:
    titles.append(i.text)

In [12]:
topic_descriptions= doc.find_all('p',{'class':"f5 color-text-secondary mb-0 mt-1"})

In [13]:
descriptions = []
for i in topic_descriptions:
    descriptions.append(i.text.strip())

In [14]:
topic_links = doc.find_all('a', {'class' : "d-flex no-underline"})

In [15]:
urls= []
for i in topic_links:
    urls.append("https://github.com"+i['href'])

In [16]:
dict_={'Titles':titles,
     'Descriptions':descriptions,
     'URLs':urls}

In [130]:
writer =  pd.ExcelWriter('data/Data.xlsx', engine = 'xlsxwriter')

In [131]:
topics_df = pd.DataFrame(dict_)

In [132]:
topics_df.to_excel(writer, sheet_name= 'Topics')

In [22]:
topics_df.to_csv('data/Topics.csv')

In [112]:

def headers(repo_tags, repo_stars, topic_name):
    username=[]
    repo_name=[]
    repo_urls=[]
    stars = []
    for i in range (len(repo_tags)):
        repo_a = repo_tags[i].find_all('a')
        username.append(repo_a[0].text.strip())
        repo_name.append(repo_a[1].text.strip())
        repo_urls.append("https://github.com/"+repo_a[1]['href'])
        k = repo_stars[i].text.strip()
        k = float(k[:-1])*1000
        stars.append(int(k))
        
    repo_dict={
    'Username' : username,
    'Repo_Name' : repo_name,
    'Stars' : stars,
    'Repo_URLs' : repo_urls
    }
    repo_df = pd.DataFrame(repo_dict)
    
    if os.path.exists(topic_name + '.csv'):
        print(topic_name + '.csv'+ " File Already Exists")
        return pd.DataFrame(repo_dict)
        return
    print("Writing {}.csv".format(topic_name))
    repo_df.to_csv(topic_name + '.csv',index=None)
    return pd.DataFrame(repo_dict)

def repos(url, name):
    response = requests.get(url)
    if response.status_code==200:
        page_contents = response.text
        t_doc = BeautifulSoup(page_contents, 'html.parser')
        repo_tags =  t_doc.find_all('h3', {'class': "f3 color-text-secondary text-normal lh-condensed"})
        repo_stars =  t_doc.find_all('a', {'class': "social-count float-none"})      
        return headers(repo_tags, repo_stars, name)
    else:
        print('Trying to reconnect to {}'.format(url))
        return repos(url, name)   
        

In [133]:
conn = sqlite3.connect('Scrapped.db')
c = conn.cursor()


In [134]:
c.execute('DROP TABLE IF EXISTS DATA')
c.execute('CREATE TABLE DATA (Topic_Name text, Username text, Repo_Name text, Stars number, Repo_URLs text)')
conn.commit()

l=len(titles)
for i in range(l):
    df_for_db = repos(urls[i], 'data/'+titles[i])
    df_for_db.to_excel(writer, sheet_name= titles[i])
    Topic_Name =[titles[i]]*l
    df_for_db['Topic_Name'] = Topic_Name
    df_for_db.to_sql('DATA', conn, if_exists='append', index = False)
    

data/3D.csv File Already Exists
data/Ajax.csv File Already Exists
data/Algorithm.csv File Already Exists
data/Amp.csv File Already Exists
data/Android.csv File Already Exists
data/Angular.csv File Already Exists
data/Ansible.csv File Already Exists
data/API.csv File Already Exists
data/Arduino.csv File Already Exists
data/ASP.NET.csv File Already Exists
data/Atom.csv File Already Exists
data/Awesome Lists.csv File Already Exists
data/Amazon Web Services.csv File Already Exists
data/Azure.csv File Already Exists
data/Babel.csv File Already Exists
data/Bash.csv File Already Exists
data/Bitcoin.csv File Already Exists
data/Bootstrap.csv File Already Exists
data/Bot.csv File Already Exists
data/C.csv File Already Exists
data/Chrome.csv File Already Exists
data/Chrome extension.csv File Already Exists
data/Command line interface.csv File Already Exists
data/Clojure.csv File Already Exists
data/Code quality.csv File Already Exists
data/Code review.csv File Already Exists
data/Compiler.csv Fi

In [135]:
if not os.path.exists('data/Data.xlsx'):
        writer.save()
        

In [118]:
c.execute('''  
SELECT * FROM DATA
          ''')

for row in c.fetchall():
    print (row)

('3D', 'mrdoob', 'three.js', 72600, 'https://github.com//mrdoob/three.js')
('3D', 'libgdx', 'libgdx', 18600, 'https://github.com//libgdx/libgdx')
('3D', 'BabylonJS', 'Babylon.js', 14400, 'https://github.com//BabylonJS/Babylon.js')
('3D', 'pmndrs', 'react-three-fiber', 13900, 'https://github.com//pmndrs/react-three-fiber')
('3D', 'aframevr', 'aframe', 12900, 'https://github.com//aframevr/aframe')
('3D', 'ssloy', 'tinyrenderer', 10900, 'https://github.com//ssloy/tinyrenderer')
('3D', 'lettier', '3d-game-shaders-for-beginners', 10700, 'https://github.com//lettier/3d-game-shaders-for-beginners')
('3D', 'FreeCAD', 'FreeCAD', 9600, 'https://github.com//FreeCAD/FreeCAD')
('3D', 'metafizzy', 'zdog', 8500, 'https://github.com//metafizzy/zdog')
('3D', 'CesiumGS', 'cesium', 7200, 'https://github.com//CesiumGS/cesium')
('3D', 'a1studmuffin', 'SpaceshipGenerator', 6800, 'https://github.com//a1studmuffin/SpaceshipGenerator')
('3D', 'timzhang642', '3D-Machine-Learning', 6800, 'https://github.com//tim

In [119]:
c.close()
conn.close()